__качаем объявления о продаже квартир с авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
place = 'sevastopol'
avito_path = place+'/kvartiry/prodam'

raw_data_path = 'data/raw/flat/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-11-04 17:05:46 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-11-04 17:05:46 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-11-04 17:05:49 | AdsListParser: downloader init
[INFO    ] 2022-11-04 17:05:49 | AvitoParser: init
[INFO    ] 2022-11-04 17:05:53 | AvitoParser: 73 pages for read
[INFO    ] 2022-11-04 17:05:53 | AdsListParser: start read and parse pages...
[INFO    ] 2022-11-04 17:05:55 | AdsListParser: read page 1
[INFO    ] 2022-11-04 17:05:58 | AdsListParser: read page 2
[INFO    ] 2022-11-04 17:06:00 | AdsListParser: read page 3
[INFO    ] 2022-11-04 17:06:03 | AdsListParser: read page 4
[INFO    ] 2022-11-04 17:07:03 | AdsListParser: read page 5
[INFO    ] 2022-11-04 17:07:08 | AdsListParser: read page 6
[INFO    ] 2022-11-04 17:07:11 | AdsListParser: read page 7
[INFO    ] 2022-11-04 17:07:14 | AdsListParser: read page 8
[INFO    ] 2022-11-04 17:07:16 | AdsListParser: read page 9
[INFO    ] 2022-11-04 17:07:19 | AdsListParser

In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

3526
3144


In [8]:
df['place']=place

In [9]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [10]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts,place
2344,2481726125,"Объявление «1-к. квартира, 44 м², 3/7 эт.» 11 фотографий",6700000,,"ул. Вакуленчука, 28","Продается 1-комнатная квартира от застройщика без комиссии. Дом сдан в эксплуатацию. В доме есть вход с двух сторон. Рядом остановка «Студгородок», торговые центры «Лаванда», «Муссон», «Ниагара». Тихий двор с детскими площадками и тренажерами. Рядом парк.",54,2022-11-04 17:11:28.840961,sevastopol
2533,2523569420,"Объявление «1-к. квартира, 31 м², 4/5 эт.» с фотографией",4600000,,"пос. городского типа Кача, ул. Авиаторов, 17","Продаётся однакомнатная квартира. \nТребует капитального ремонта. При необходимости, окажем услуги по отделочным работам под ключ. Просмотр в любое время по предварительной договорённости.",58,2022-11-04 17:11:28.840961,sevastopol


In [11]:
# !pip install dash_table